In [1]:
import warnings
warnings.filterwarnings("ignore")

import sys
sys.path.append("../../")

from experiment import Exp

from dataloaders import data_set,data_dict
import torch
import yaml
import os

import torch
from ptflops import get_model_complexity_info

In [2]:
torch.__version__

'1.13.1+cu117'

# Parameter settings

# Training parameters

In [3]:
class dotdict(dict):
    """dot.notation access to dictionary attributes"""
    __getattr__ = dict.get
    __setattr__ = dict.__setitem__
    __delattr__ = dict.__delitem__

args = dotdict()   
# TODO change the path as relative path
args.to_save_path     = r"../../Run_logs"            
args.freq_save_path   = r"../../Freq_data"
args.window_save_path = r"../../Sliding_window"
args.root_path        = r"../../datasets"


args.drop_transition  = False
args.datanorm_type    = "standardization" # None ,"standardization", "minmax"


args.batch_size       = 256                                                     
args.shuffle          = True
args.drop_last        = False
args.train_vali_quote = 0.90                                           


# training setting 
args.train_epochs            = 150

args.learning_rate           = 0.001  
args.learning_rate_patience  = 7
args.learning_rate_factor    = 0.1


args.early_stop_patience     = 15

args.use_gpu                 = True if torch.cuda.is_available() else False
args.gpu                     = 0
args.use_multi_gpu           = False

args.optimizer               = "Adam"
args.criterion               = "CrossEntropy"

## Data parameters



In [4]:
# oppo : 25/50  2
# dg   : 12/24  4
# dsads: 12/24  4
# pamap2: 10/20  5
# wisdm: 12/24  4
# skodar : 12/24 4

In [5]:
args.seed                             = 1


args.data_name                        =  "dg"

args.wavelet_filtering                = False
args.wavelet_filtering_regularization = False
args.wavelet_filtering_finetuning     = False
args.wavelet_filtering_finetuning_percent = 0.5
args.wavelet_filtering_learnable      = False
args.wavelet_filtering_layernorm      = False

args.regulatization_tradeoff          = 0
args.number_wavelet_filtering         = 12


args.difference       = False 
args.filtering        =  False
args.magnitude        =  False
args.weighted_sampler = False




args.pos_select       = None
args.sensor_select    = None


args.representation_type = "time"
args.exp_mode            = "LOCV"
if args.data_name      ==  "skodar":
    args.exp_mode            = "SOCV"
config_file = open('../../configs/data.yaml', mode='r')
data_config = yaml.load(config_file, Loader=yaml.FullLoader)
config = data_config[args.data_name]

args.root_path       = os.path.join(args.root_path,config["filename"])
args.sampling_freq   = config["sampling_freq"]
args.num_classes     =  config["num_classes"]
window_seconds       = config["window_seconds"]
args.windowsize      =   int(window_seconds * args.sampling_freq) 
args.input_length    =  args.windowsize
# input information
args.c_in            = config["num_channels"]


if args.difference:
    args.c_in = args.c_in*2

if args.wavelet_filtering :
    
    if args.windowsize%2==1:
        N_ds = int(torch.log2(torch.tensor(args.windowsize-1)).floor()) - 2
    else:
        N_ds = int(torch.log2(torch.tensor(args.windowsize)).floor()) - 2

    args.f_in            =  args.number_wavelet_filtering*N_ds+1
else:
    args.f_in            =  1



## Model parameters

In [6]:
# dsads 27838817.0 99367   26
# skodar  9907786.0   67092 24
# wisdm 733382.0 16
# dg  1041602.0 16
# oppo 7377290.0 28

In [7]:
# 如果我们设置为 


args.model_type              = "tinyhar"#"deepconvlstm"#"sahar" #"deepconvlstm"

args.cross_channel_interaction_type = "attn"
args.cross_channel_aggregation_type = "FC"
args.temporal_info_interaction_type = "lstm"
args.temporal_info_aggregation_type = "tnaive"

exp = Exp(args)

Use GPU: cuda:0
Build the TinyHAR model!
Done!
Parameter : 29664
Set the seed as :  1


In [8]:
macs, params = get_model_complexity_info(exp.model, (1  ,args.input_length,args.c_in), as_strings=False,
                                       print_per_layer_stat=True, verbose=False)
print('{:<30}  {:<8}'.format('Computational complexity: ', macs))
print('{:<30}  {:<8}'.format('Number of parameters: ', params))

model_builder(
  29.66 k, 99.993% Params, 1.59 MMac, 100.000% MACs, 
  (model): TinyHAR_Model(
    29.66 k, 99.993% Params, 1.59 MMac, 100.000% MACs, 
    (layers_conv): ModuleList(
      6.34 k, 21.373% Params, 1.26 MMac, 79.109% MACs, 
      (0): Sequential(
        160, 0.539% Params, 97.2 KMac, 6.113% MACs, 
        (0): Conv2d(120, 0.405% Params, 64.8 KMac, 4.075% MACs, 1, 20, kernel_size=(5, 1), stride=(1, 1))
        (1): ReLU(0, 0.000% Params, 10.8 KMac, 0.679% MACs, inplace=True)
        (2): BatchNorm2d(40, 0.135% Params, 21.6 KMac, 1.358% MACs, 20, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): Sequential(
        2.06 k, 6.944% Params, 524.16 KMac, 32.966% MACs, 
        (0): Conv2d(2.02 k, 6.810% Params, 509.04 KMac, 32.015% MACs, 20, 20, kernel_size=(5, 1), stride=(2, 1))
        (1): ReLU(0, 0.000% Params, 5.04 KMac, 0.317% MACs, inplace=True)
        (2): BatchNorm2d(40, 0.135% Params, 10.08 KMac, 0.634% MACs, 20, eps=1e-05, momentum=

exp.

In [9]:
exp.train()

 ----------------------- load all the data -------------------
-----------------------Sliding file are generated -----------------------
-----------------------Sliding file are generated -----------------------
================ LOCV Mode ====================
================ 10 CV ======================
================ the 0 th CV Experiment ================ 
Leave one Out Experiment : The 1 Part as the test
[-] Target sampling weights:  [3.88485296e-05 3.41997264e-04]
Train data number :  28665
The number of classes is :  2
The input_length  is :  64
The channel_in is :  9
Validation data number :  3185
Test data number :  20267
================ Build the model ================ 
Build the TinyHAR model!
Epoch: 1 cost time: 15.246124505996704
VALI: Epoch: 1, Steps: 112 | Train Loss: 0.2402029  Vali Loss: 0.1831981 Vali Accuracy: 0.9161695  Vali weighted F1: 0.9181879  Vali macro F1 0.7793566 
Validation loss decreased (inf --> 0.183198).  Saving model ...
Epoch: 2 cost time: 14.664674

KeyboardInterrupt: 

#### if self.args.model_type == "mcnn":
            config_file = open('../../configs/model.yaml', mode='r')
            config = yaml.load(config_file, Loader=yaml.FullLoader)["mcnn"]
            setting = "mcnn_data_{}_seed_{}_windowsize_{}_waveFilter_{}_Fscaling_{}_cvfilter_{}_Regu_{}_wavelearnble_{}".format(self.args.data_name,
                                                                                                                                              self.args.seed,
                                                                                                                                              self.args.windowsize,
                                                                                                                                              self.args.wavelet_filtering,
                                                                                                                                              self.args.filter_scaling_factor,
                                                                                                                                              config["nb_filters"],
                                                                                                                                              self.args.wavelet_filtering_regularization,
                                                                                                                                              self.args.wavelet_filtering_learnable )
            return setting


pamap2:
  filename: "PAMAP2_Dataset/Protocol"
  sampling_freq: 33
  num_classes: 12
  num_channels: 18
  window_seconds: 5.12
  
self.model  = MCNN((1,f_in, self.args.input_length, self.args.c_in ), 
                                self.args.num_classes,
                                self.args.filter_scaling_factor,
                                config)
(1,1,169,18) 12, 1
f_in = 1
args.windowsize      =   int(window_seconds * args.sampling_freq) 
                     = 169
args.filter_scaling_factor =1                     

args.root_path       = os.path.join(args.root_path,config["filename"])
args.sampling_freq   = config["sampling_freq"]
args.num_classes     =  config["num_classes"]
window_seconds       = config["window_seconds"]
args.windowsize      =   int(window_seconds * args.sampling_freq) 
args.input_length    =  args.windowsize
# input information
args.c_in            = config["num_channels"]

In [ ]:
from models.TinyHAR import TinyHAR_Model
from brevitas.export import export_onnx_qcdq

The number of classes is :  2
The input_length  is :  64
The channel_in is :  9

In [ ]:
num_classes = 2
num_channels = 9
windowsize = 64
input_shape =(1, 1, windowsize, num_channels)
print(input_shape)
config_file = open('../../configs/model.yaml', mode='r')
config = yaml.load(config_file, Loader=yaml.FullLoader)["tinyhar"]


In [ ]:
model = TinyHAR_Model(input_shape = input_shape,
                      number_class = num_classes,
                      filter_num = 20,
                      cross_channel_interaction_type = "attn",
                      cross_channel_aggregation_type = "FC",
                      temporal_info_interaction_type = "conv",
                      temporal_info_aggregation_type = "tnaive")

In [ ]:
loadPath = "/home/bian/finn/notebooks/HAR/Run_logs/logs/tinyhar_data_dg_seed_1_windowsize_64_cvfilter_20_CI_attn_CA_FC_TI_conv_TA_tnaive/cv_0/best_vali.pth"

model.load_state_dict(torch.load(loadPath),strict=False)
model.eval()

In [ ]:
import torch
import brevitas.onnx as bo
from brevitas.quant_tensor import QuantTensor

ready_model_filename = "daphnet_attn_conv_4.onnx"

# Export to ONNX
#Move to CPU before export
model.cpu()
inp = torch.randn(1, 1, windowsize, num_channels)
bo.export_finn_onnx(model, inp, ready_model_filename)
"""
torch.onnx.export(model, 
                  data_usage, 
                  ready_model_filename,   # where to save the model (can be a file or file-like object)
                  export_params=True,        # store the trained parameter weights inside the model file         # the ONNX version to export the model to
                  do_constant_folding=True,  # whether to execute constant folding for optimization
                  input_names = ['input'],   # the model's input names
                  output_names = ['output'] # the model's output names
                  )
"""
print("Model saved to %s" % ready_model_filename)

In [ ]:
from LatencyMeasurer import LatencyMeasurer

model = model.cpu()
measurer = LatencyMeasurer(model)

cpu_latency = measurer.measure_latency_CPU(input_shape)
cpu_finn_latency = measurer.measure_CPU_latency_finn(model.forward_truncated, input_shape)
print(f"cpu latency for quantized TinyHAR: {cpu_latency}s,\nfinn part latency: {cpu_finn_latency}s")
model = model.cuda()
measurer = LatencyMeasurer(model)
gpu_latency = measurer.measure_GPU_latency(input_shape)
gpu_finn_latency = measurer.measure_GPU_finn_latency(model.forward_truncated, input_shape)
print(f"gpu latency for quantized TinyHAR: {gpu_latency}s,\nfinn part latency: {gpu_finn_latency}s")

In [ ]:
import PowerMeasurer

model = model.cuda()
gpu_power = PowerMeasurer.measure_gpu_power(model, input_shape)
gpu_power_finn = PowerMeasurer.measure_gpu_finn_power(model, model.forward_truncated,input_shape)
print(f"gpu power consumption for quantized TinyHAR: {gpu_power}W,\nfinn part latency: {gpu_power_finn}w")

# 实验

In [ ]:
# 如果我们设置为 


args.model_type              = "tinyhar"#"deepconvlstm"#"sahar" #"deepconvlstm"

args.cross_channel_interaction_type = "attn"
args.cross_channel_aggregation_type = "FC"
args.temporal_info_interaction_type = "lstm"
args.temporal_info_aggregation_type = "tnaive"

exp = Exp(args)

In [ ]:
exp.model

In [ ]:
exp.train()

In [ ]:
dataset = data_dict[args.data_name](args)

In [ ]:
import numpy as np

In [ ]:
start_index = dataset.train_slidingwindows[0][1]
end_index   = dataset.train_slidingwindows[0][2]
sample_x_1    = dataset.data_x.iloc[start_index:end_index, 1:-1].values

start_index = dataset.train_slidingwindows[100][1]
end_index   = dataset.train_slidingwindows[100][2]
sample_x_2    = dataset.data_x.iloc[start_index:end_index, 1:-1].values

temp_1 = np.expand_dims(sample_x_1,0)
temp_2 = np.expand_dims(sample_x_2,0)
combined_x = np.concatenate([temp_1,temp_2],axis=0)
combined_x = np.expand_dims(combined_x,1)
combined_x = torch.tensor(combined_x).double().to(exp.device)
print(combined_x.shape)

In [ ]:
out = exp.model(combined_x)
out = out.cpu().detach().numpy()

In [ ]:
out.shape

In [ ]:
weight = exp.model.wave_conv.wavelet_conv.weight.cpu().detach().numpy()

In [ ]:
for n,p in exp.model.named_parameters():
    print(n)

In [ ]:
import matplotlib.pyplot as plt
index1 = 0
index2 = 0

plt.figure()
plt.plot(out[0,0,:,index2])
plt.plot(out[0,1,:,index2])
plt.plot(out[0,2,:,index2])
plt.plot(out[0,3,:,index2])

In [ ]:
# 如果我们设置为 
# args.wavelet_filtering_learnable      = True
# exp = Exp(args)